Must have matplotlib==3.1.3, otherwise Python won't show the last plot (it'll give an error caused by Matplotlib's latest release)

### Import Libraries and Configuration Parameters

In [1]:
import exoplanet as xo
from astropy.timeseries import LombScargle
from scipy import interpolate, signal, fft
from scipy.interpolate import interp1d

import numpy as np
import seaborn as sns
import pandas as pd
import os, sys

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import lightkurve as lk
from astropy.units import cds

%matplotlib inline

In [2]:
from pylab import rcParams

rcParams.update({
    "figure.dpi": 200,
    "savefig.dpi": 300,
    "axes.linewidth": 2,
    "lines.linewidth": 2, 
    "figure.figsize": (16,6), 
    "text.usetex": False, 
    "axes.titlesize": 30,
    "axes.labelsize": 25, ## fontsize of the x any y labels
    "xtick.labelsize": 20,
    "ytick.labelsize": 20
})

In [3]:
kepler_tref = 2454833
lit_period = 2.18909730
KIC = 4544587
lit_t0 = 121.14 

In [4]:
segment = "0_43978"
lc = pd.read_csv("KIC%s/%s/tfreq_analysis/lc_original.txt" %(KIC, segment), sep=" ", header=1, names=["Time", "Flux"])
x, y = lc["Time"], lc["Flux"]

FileNotFoundError: [Errno 2] File KIC4544587/0_43978/tfreq_analysis/lc_original.txt does not exist: 'KIC4544587/0_43978/tfreq_analysis/lc_original.txt'

In [ ]:
def read_txt(path_file, sep = " ", header = None, names = None):
    df =  pd.read_csv(path_file, sep=" ", header = header, names = names)
    return df

### Read the Data 

In [ ]:
osc_data = pd.read_csv("KIC%s/%s/tfreq_analysis/osc_results.csv" %(KIC, segment), header=0, sep=",") 
osc_model = read_txt("KIC%s/%s/tfreq_analysis/osc_model.txt" %(KIC, segment), header = 1, names=["Time", "Flux"]);
spectrogram_lc = read_txt("KIC%s/%s/tfreq_analysis/osc_and_noise_models.txt" %(KIC, segment), header=1, names=["Time", "Flux"])
noise_model = read_txt("KIC%s/%s/tfreq_analysis/noise_model.txt" %(KIC, segment), header=1, names=["Time", "Flux"])
time, flux = spectrogram_lc["Time"].values, spectrogram_lc["Flux"].values
time_osc, flux_osc = osc_model["Time"].values, osc_model["Flux"].values

fig, ax = plt.subplots(1,1, figsize=(16,6))
ax.scatter(time, flux, marker=".", s=1)
ax.set_title("Observed Flux - (Phase Curve Model + Eclipse Model)", y=1.02, fontsize=25)
ax.set_xlabel("Time - %i [BKJD days]"%kepler_tref, fontsize=20)
ax.set_ylabel("Raw Flux (ppt)", fontsize=20);


### Check if the Data is Evenly Sampled

In [ ]:
def uneven_to_evenly_sampled_data(t, f, threshold):
    t = np.sort(t)
    dt_points = (t[1:] - t[:-1])
    outliers, outliers_idx = dt_points[dt_points > threshold], np.where(dt_points > threshold)[0]
    dt_median = np.median(dt_points)
    npoints_nodata = outliers/dt_median

    # Let's plot the outliers
    fig, ax = plt.subplots(1,1, figsize=(16,6))
    ax.scatter(t[:-1], dt_points, marker="o", s=1, c="lightgrey")
    ax.scatter(t[:-1][outliers_idx], outliers, marker="o", c="crimson", s=20, label="Outliers")
    ax.legend(fontsize=18)
    ax.set_title("Time between Observations")
    ax.set_xlabel("Time - %i [BKJD days]"%kepler_tref, fontsize=20)
    ax.set_ylabel("$\Delta t$ [d]", fontsize=20);
    
    #Let's fill the gaps
    fake_flux_function = interp1d(t, f)
    new_t = np.array(np.copy(t))
    new_f = np.array(np.copy(f))
    print("Original Length of Time Array = %i" %len(t))

    fig, ax = plt.subplots(1,1)
    ax.scatter(t, f, marker="o", s=2, c="lightsteelblue", label="Original Flux Points")
    for i,idx in enumerate(outliers_idx):
        nfake_points = round(npoints_nodata[i])-1
        t_init = t[idx]
        t_end = t[idx+1]
        x_fake_flux =  np.linspace(t_init, t_end, num = nfake_points+2)[1:-1]
        y_fake_flux = fake_flux_function(x_fake_flux)
        ax.scatter(x_fake_flux, y_fake_flux, marker="o", color="red")
        new_t = np.append(new_t, x_fake_flux)
        new_f = np.append(new_f, y_fake_flux)

    ax.scatter([], [], c="red", label="Fake Flux Points")
    ax.set_xlabel("Time - %i [BKJD days]"%kepler_tref)
    ax.set_ylabel("Raw Flux (ppt)")
    ax.legend(fontsize=18)
        
    print("Final Length of Time Array = %i" %len(new_t))
    print("Added %i points." %(len(new_t)-len(t)))

    
    #Let's plot dt again to confirm that we've added fake flux points correctly.
    sidx = np.argsort(new_t)
    new_t = new_t[sidx]
    new_f = new_f[sidx]
    
    dt_points = (new_t[1:] - new_t[:-1])
    fig, ax = plt.subplots(1,1)
    ax.scatter(new_t[:-1], dt_points, s=15, c="grey")
    ax.set_title("Time between Observations")
    ax.set_xlabel("Time - %i [BKJD days]"%kepler_tref)
    ax.set_ylabel("$\Delta t$ [d]");
    
    print("Range of time gaps = {:>.3e} days ({:>.3e} times median spacing)".format((np.max(dt_points) - np.min(dt_points)), (np.max(dt_points) - np.min(dt_points))/np.median(dt_points)))

    plt.pause(0.01)
    plt.close("all")
    
    return outliers, outliers_idx, new_t, new_f

In [ ]:
outlier_threshold = 0.001
outliers, outliers_idx, new_time, new_flux = uneven_to_evenly_sampled_data(time, flux, outlier_threshold)

In [ ]:
old_time = np.copy(time)
old_flux = np.copy(flux)

time = np.copy(new_time)
flux = np.copy(new_flux)

### Get Signal Properties

In [ ]:
def get_signal_properties(signal, print_res = False):
    num_samples = len(flux) #number of samples
    sampling_inteval = np.median(np.diff(time)) #sampling interval (or "rate"). Time span between consecutive samples [d]
    sampling_frequency = 1.0/sampling_inteval # Sampling frequency [1/d] 
    nyquist_frequency = sampling_frequency/2.0
    total_duration = (num_samples - 1) / sampling_frequency #duration of the signal

    if print_res: 
        print("Nº of samples = %i" %num_samples)
        print("Sampling Interval = %0.08f [d]" %sampling_inteval)
        print("Total Duration of Signal = %0.02f [d]" %total_duration)
        print("Sampling frequency = %0.02f [d^-1]" %sampling_frequency)
        print("Nyquist frequency = %0.02f [d^-1]" %nyquist_frequency)

    return num_samples, sampling_inteval, sampling_frequency, nyquist_frequency, total_duration

Nt, dt, fs, f_nyquist, time_span = get_signal_properties(flux, print_res = True)

### Generate Grid of Frequencies and Run LS Periodogram

We use a LS periodogram because our data is unevenly sampled.

In [ ]:
def ls_estimator(t, f, frequencies, filter_period=None, npeaks=15, normalization="psd"):
    #Function based on "exoplanet"'s' LS estimator code but adjusted for a variable frequency grid
    #(https://github.com/exoplanet-dev/exoplanet/blob/main/src/exoplanet/estimators.py)
    
    # Estimate the power spectrum
    periods = 1. / frequencies
    model = LombScargle(t, f)
    powers = model.power(frequencies, method="fast", normalization=normalization)
    powers /= len(t)
    power_est = np.array(powers)
    
    # Identify Peaks
    peaks = xo.find_peaks(frequencies, power_est, max_peaks=npeaks)

    return dict(periodogram=(frequencies, power_est), peaks=peaks, ls=model)
    
orbital_frequency = 1/lit_period
minfr = 5*orbital_frequency #[d^-1] or 1/(total_duration) #5*orbital freq is about 10 hours
maxfr = 200 #[d^-1] #200 d^-1 is about 7 min
N_samples_per_peak = 10 #shouldn't be less than 5. 10 often used as a conservative measure to have good spectral resol'n (p. 36 vanderplas)
df = 1/(time_span*N_samples_per_peak) #p. 36 vanderplas
grid_resolution = (maxfr-minfr)/df+1
freq_grid = np.linspace(minfr, maxfr, int(grid_resolution))

ls_spec = ls_estimator(time, flux, freq_grid, normalization="psd")
ls_spec_old = ls_estimator(old_time, old_flux, freq_grid, normalization="psd")

fig, ax = plt.subplots(1,1, figsize=(14,6.5))
ax.plot(ls_spec_old["periodogram"][0], ls_spec_old["periodogram"][1], label="Original LC",c="lightsteelblue")
#ax.plot(ls_spec["periodogram"][0], ls_spec["periodogram"][1], ms=0.5, alpha=0.5, c="orange", label = "LC with Injected Fake Flux points")
ax.set_xlabel(r"Frequency [$d^{-1}$]", size=20)
ax.set_ylabel("Power")
ax.set_title("LS Periodogram")
ax.set(xlim=(0, 1.2*max(osc_data.freq)))
ax.text(0.1,0.6, "Low Frequency g-mode regime", fontsize=15, transform=ax.transAxes, bbox=dict(facecolor='red', alpha=0.4));
ax.text(0.62,0.6, "High Frequency p-mode regime", fontsize=15, transform=ax.transAxes, bbox=dict(facecolor='green', alpha=0.4));
#ax.legend(fontsize=15)

#ax.set(xlim=(min(freq_grid),5));

fig.tight_layout()
fig.savefig("KIC%s/frequency_analysis/lsperiodogram.png"%KIC, dpi=300)

If the data were evenly sampled, the LS periodogram with a "PSD" normalization would be equivalent to the Fourier Transform (provided no uncertainties are given). As the cells below show, the results of our LS Periodogram are not quite the same as those from the FFT because the data is not evenly sampled. More information can be found here: https://docs.astropy.org/en/stable/timeseries/lombscargle.html#lomb-scargle-normalization

In [ ]:
import astropy.units as u
def fourier_periodogram(t, y):
    N = len(t)
    frequency = np.fft.fftfreq(N, t[1] - t[0])
    y_fft = np.fft.fft(y)
    positive = (frequency > 0)
    return frequency[positive], (1. / N) * abs(y_fft[positive]) ** 2

#Our dataset -- Not Evenly Sampeld
frequency_fourier, PSD_fourier = fourier_periodogram(time, flux)
ls = LombScargle(time, flux) 
PSD_LS = ls.power(frequency_fourier, normalization="psd")
print(u.allclose(PSD_fourier, PSD_LS)) 

fig, ax = plt.subplots(2,1, figsize=(12,8), sharex=True)
fig.subplots_adjust(hspace=0.0)
ax[0].plot(frequency_fourier, PSD_fourier, label = "FFT")
ax[0].plot(frequency_fourier, PSD_LS, alpha = 0.5, label = "LS")
ax[0].set_title("Our Dataset", fontsize=20)
ax[0].legend() 
ax[0].set_xlabel(r"Frequency [d$^{-1}$]")
ax[0].set_ylabel("Power", fontsize=20)
#ax[0].set_ylim(0,1)

ax[1].plot(frequency_fourier, (PSD_fourier-PSD_LS)/PSD_LS)
ax[1].set_xlim(0, maxfr)
ax[1].set_xlabel(r"Frequency [d$^{-1}$]")
ax[1].set_ylabel("Power", fontsize=20)
#ax[1].set_ylim(0, 4000)
#fig.savefig("LS_vs_FFT.png")

In [ ]:
lc = lk.LightCurve(time = time, flux = flux)
pg = lc.to_periodogram(frequency = np.arange(min(freq_grid), f_nyquist, 10E-3), 
                       method='lombscargle', 
                       normalization='amplitude')

fig,ax = plt.subplots(1,3, figsize=(25, 5))
fig.subplots_adjust(wspace=0.25)
pg.plot(scale='log', ax=ax[0]);
ax[0].set_xlabel(r"Frequency [d$^{-1}$]")
ax[0].set_ylabel("Power")
ax[0].set_xticks([2, 5, 10, 20, 40, 100, 300, int(f_nyquist)])
#ax[0].set_xlim([int(min(freq_grid)), int(max(freq_grid))])
ax[0].get_xaxis().set_major_formatter(ScalarFormatter())

pg.plot(scale='log', ax=ax[1]);
ax[1].set_xlabel(r"Frequency [d$^{-1}$]")
ax[1].set_ylabel("Power")
ax[1].set_xticks([2, 3, 4, 6, 10, 20])
ax[1].set_xlim([int(min(freq_grid)), 20])
ax[1].get_xaxis().set_major_formatter(ScalarFormatter())

pg.plot(scale='log', ax=ax[2]);
ax[2].set_xlabel(r"Frequency [d$^{-1}$]")
ax[2].set_ylabel("Power")
ax[2].set_xticks([30, 40, 50, 60])
ax[2].set_xlim([30, 60])
ax[2].get_xaxis().set_major_formatter(ScalarFormatter())

In [ ]:
def plot_spectrum(freq, flux_fft, freq_lims = None, amp_lims = None):
    if freq_lims is None: 
        freq_min = 0.0
        freq_max = np.max(freq)
    
    else:
        freq_min, freq_max = freq_lims
        
    print(freq_min, freq_max)
    fig = plt.figure(figsize = (16, 8))
    ax = plt.gca()
    ax.plot(freq, np.abs(flux_fft), linewidth = 1, color = 'black')
    ax.set_xlabel('Frequency (day$^{-1}$)')
    ax.set_ylabel('Amplitude (ppm day)')
    ax.set_yscale('log')
    ax.set_xlim(freq_lims)
    if amp_lims is not None: 
        ax.set_ylim(amp_lims)
    #plt.savefig(dir_fig+"spectrum_%ito%i.png"% (freq_min, freq_max), dpi=300)
    
def plot_raw(time, flux, freq) :
    #Plot raw time series and Fourier transform.
    
    freq_lims, amp_lims, name = (None, [1.0E-1, 1.0E4], 'spectrum_raw_full')        # Whole spectrum.
    plot_spectrum(freq, flux_fft, amp_lims = amp_lims, freq_lims = freq_lims)
    
    freq_lims, amp_lims, name = ([0.0, 15.0], [2.0E0, 3.0E4], 'spectrum_raw_00_15') # First peak.
    plot_spectrum(freq, flux_fft, amp_lims = amp_lims, freq_lims = freq_lims)
    
    freq_lims, amp_lims, name = ([30.0, 55.0], [2.0E0, 3.0E4], 'spectrum_raw_30_55')# Second peak.
    plot_spectrum(freq, flux_fft, amp_lims = amp_lims, freq_lims = freq_lims)
    
    return

def get_fft_and_freq(time, flux):

    # Calculate the discrete Fourier transform and associated frequencies.
    flux_fft = np.fft.rfft(flux)
    freq_fft = np.fft.rfftfreq(Nt, d = dt)

    return freq_fft, flux_fft

freq_fft, flux_fft = get_fft_and_freq(time, flux)
plot_raw(time, flux, freq_fft)

In [ ]:
def plot_filtering_freq_domain_magnitude(freq, flux_fft, flux_fft_filtered, filter_, filter_lower_freq, filter_upper_freq):
    #Source: Harry's code
    
    fig, ax_arr = plt.subplots(2, 1, figsize = (14, 10), gridspec_kw = {'height_ratios': [1, 3]}, sharex = True)
    ax = ax_arr[0] 
    
    # Plot the magnitude of the filter.
    ax.plot(freq, np.abs(filter_), c = 'g', label = 'Filter', zorder = 10)
    ax.legend(loc = 'upper right')

    ax.set_ylim([0.0, 1.05])
    ax = ax_arr[1]

    # Plot the flux.
    flux_kwargs = {'alpha' : 0.7, 'linewidth' : 1}
    abs_flux_fft = np.abs(flux_fft)
    i_freq_allowed = np.where(freq > filter_lower_freq)[0]
    max_abs_flux_fft = np.max(abs_flux_fft[i_freq_allowed])
    flux_fft_normalised = flux_fft/max_abs_flux_fft
    #ax.plot(freq, np.abs(flux_fft_normalised), label = 'Flux before filtering', **flux_kwargs) 
    ax.plot(freq, np.abs(flux_fft), label = 'Flux before filtering', **flux_kwargs) 

    # Plot the filtered flux.
    abs_flux_fft_filtered = np.abs(flux_fft_filtered)
    max_abs_flux_fft_filtered = np.max(abs_flux_fft_filtered)
    flux_fft_filtered_normalised = flux_fft_filtered/max_abs_flux_fft_filtered
    #ax.plot(freq, np.abs(flux_fft_filtered_normalised), label = 'Flux after filtering', **flux_kwargs)
    ax.plot(freq, np.abs(flux_fft_filtered), label = 'Flux after filtering', **flux_kwargs)

    ax.set_xlabel('Frequency (cycles per day)')
    ax.set_ylabel('Spectral amplitude (ppm day)')
    ax.set_yscale('log')
    ax.set_xlim([freq[0], freq[-1]])
    
    amp_lims = [1E-3, 2.0E4]
    ax.set_ylim(amp_lims)
    ax.set_xlim([0.0, 15.0])
    ax.legend()

    # Plot the cutoff frequency.
    for ax in ax_arr:
        for freq_ in [filter_lower_freq, filter_upper_freq]:
            ax.axvline(freq_, color = 'k', zorder = 9, alpha = 0.5)

    plt.tight_layout()
    #path_fig = os.path.join(dir_fig, 'filtering_freq.png')
    #print('Saving to {:}'.format(path_fig))
    #plt.savefig(path_fig, dpi = 300)
    return

def plot_filtering_time_domain(time, flux, time_downsampled, flux_filtered_downsampled, raw_only = False, d_t = 1.0):
    time_span = time[-1] - time[0]
    abs_flux = np.abs(flux)
    flux_span = np.max(abs_flux)- np.min(abs_flux) 
    offset = 0.8*flux_span

    n_traces = np.ceil(time_span/d_t)
    n_traces_int = int(n_traces)

    #n_subplots = 3
    n_traces = 3 
    n_traces_float = float(n_traces)
    
    #n_subplots_float = float(n_subplots)
    #fig, ax_list = plt.subplots(n_subplots, 1, figsize = (11.0, 8.5))
    fig = plt.figure(figsize = (14, 10))
    ax = plt.gca()
    flux_kwargs = {'alpha' : 0.7, 'linewidth' : 1}

    #label_flux = 'Raw data'
    label_flux = 'After de-trending'
    label_flux2 = 'After filtering\nand downsampling'
    
    shift = 0
    for i in range(n_traces_int):
        t0 = time[0] + (i*d_t)
        t1 = time[0] + ((i + 1)*d_t)

        j = ((time >= t0) & (time <= t1))
        k = ((time_downsampled >= t0) & (time_downsampled <= t1))

        i_offset = -i*offset

        #flux_mean = np.mean(flux[j])
        if i == 0:
            label_flux = label_flux
            label_flux2 = label_flux2
        else:
            label_flux = None
            label_flux2 = None

        ax.plot(time[j] - t0, flux[j] + i_offset + shift, c = 'dodgerblue', label = label_flux, **flux_kwargs)
        
        if not raw_only:
            ax.plot(time_downsampled[k] - t0, flux_filtered_downsampled[k] + i_offset + shift, c= 'k', label = label_flux2, **flux_kwargs)


    ax.legend(loc = 'lower right', fontsize=20)
    ax.set_xlabel('Time (days)')
    #ax.set_ylabel('Flux (ppm)')
    ax.set_xlim([0.0, d_t])
    ax.set_ylim([-(n_traces_int + 0.5)*offset - 3.5, 2.0])
    
    for axis in ['right', 'left', 'top']:
        ax.spines[axis].set_visible(False)

    ax.set_yticks([])
    plt.tight_layout()
    
    if raw_only:
        name_fig = 'flux_gather_raw'

    else:
        name_fig = 'flux_gather_filtered'
        
    #path_fig = os.path.join(dir_fig, '{:}.png'.format(name_fig))
    #print('Saving to {:}'.format(path_fig))
    #plt.savefig(path_fig, dpi = 300)

    return


def filter_flux(time, flux, freq):
    # Create a fourth-order zero-phase Butterworth filter.
    # A filter with no phase shift is created by applying the filter
    # forward and backwards, which doubles the original order of the filter.
    
    filter_half_order = 2 
    filter_lower_freq =  1.0 # Lower cut-off frequency, cycles per day.
    filter_upper_freq = maxfr #140.0 # Upper cut-off frequency, cycles per day
    
    # Filter cut-off frequencies in radians per day.
    filter_angular_freqs = 2.0*np.pi*np.array([filter_lower_freq, filter_upper_freq])
    
    # Get the filter coefficients.
    filter_numerator, filter_denominator = signal.butter(filter_half_order, filter_angular_freqs, 'bandpass', analog = True)

    # Evaluate the filter at the frequencies of the FFT.
    _, filter_ = signal.freqs(filter_numerator, filter_denominator, worN = 2.0*np.pi*freq)
    
    # Square the filter to get the effect of applying the filter backwards as well.
    filter_ = np.abs(filter_)**2.0

    flux_fft_filtered = filter_*flux_fft # Apply the filter to the FFT.
    
    # Reconstruct the filtered signal with inverse FFT.
    flux_filtered = np.fft.irfft(flux_fft_filtered, len(flux))
    
    # Down-sample the filtered signal.
    # freq_sampling:  Sampling frequency (cycles per day).
    # freq_sampling_downsampled_appx
    #   Approximate sampling frequency after downsampling (samples per day).
    #   This is chosen to be twice the upper frequency of the bandpass filter,
    #   so that the frequency content within the filter limits is adequately
    #   sampled.
    # sampling_interval_downsampled_appx : Approximate sampling interval after downsampling (days).
    # n_samples_downsampled : Number of samples in the time domain after downsampling.
    # sampling_interval_downsampled_appx : Sampling interval after downsampling (days).
    # time_downsampled : Times of the sample points (days) after downsampling.
    # n_samples_downsampled: Number of points (in the time domain) after down-sampling.
   
    freq_sampling_downsampled_appx = 2.0*filter_upper_freq 
    sampling_interval_downsampled_appx = 1.0/freq_sampling_downsampled_appx
    n_samples_downsampled = int(np.round(time_span/sampling_interval_downsampled_appx)) + 1
    time_downsampled = np.linspace(time[0], time[-1], num = n_samples_downsampled)
    sampling_interval_downsampled = time_span/(n_samples_downsampled - 1)
    freq_downsampled = 1.0/sampling_interval_downsampled
    flux_filtered_downsampled = np.interp(time_downsampled, time, flux_filtered)
    
    # Print sampling info.
    print('Original sample frequency:\t{:>10.2f} samples per day'.format(fs))
    print('Final sample frequency:\t\t{:>10.2f} samples per day'.format(freq_downsampled))

    # Save the down-sampled signal.
    fig, ax = plt.subplots(1,1, figsize=(14,6))
    ax.scatter(time, flux, s=5, label="Original Signal")
    ax.scatter(time_downsampled,flux_filtered_downsampled, s=5, label="Downsampled Signal")
    ax.legend(fontsize=20)
    ax.set_xlabel('Time (days)')
    ax.set_ylabel('Raw Flux (ppm)')
    #plt.tight_layout()
    #plt.savefig(dir_fig+"raw_vs_downsampled_signal.png")
    
    #array_out = np.array([time_downsampled, flux_filtered_downsampled])
    #path_out = os.path.join(dir_fig, 'flux_filtered.txt') 
    #print('Saving to {:}'.format(path_out))
    #np.save(path_out, array_out)
    
    # Also create a high--pass-only (i.e de-trending) version of the filter (for plotting only).
    filter_highpass_numerator, filter_highpass_denominator = signal.butter( filter_half_order, filter_angular_freqs[0], 'high', analog = True)
   
    # Evaluate the filter at the frequencies of the FFT.
    _, filter_highpass = signal.freqs(filter_highpass_numerator, filter_highpass_denominator, worN = 2.0*np.pi*freq)
    
    # Square the filter to get the effect of applying the filter backwards # as well.
    filter_highpass = np.abs(filter_highpass)**2.0

    # Apply the filter to the FFT.
    flux_fft_highpassed = filter_highpass*flux_fft

    # Reconstruct the filtered signal
    flux_highpassed = np.fft.irfft(flux_fft_highpassed, len(flux))

    # Plot.
    plot_filtering_time_domain(time, flux_highpassed, time_downsampled, flux_filtered_downsampled, raw_only = False, d_t = 1.0)
    plot_filtering_freq_domain_magnitude(freq, flux_fft, flux_fft_filtered, filter_, filter_lower_freq, filter_upper_freq)

    return time_downsampled, flux_filtered_downsampled, freq_downsampled

In [ ]:
time_downfiltered, flux_downfiltered, freq_sampling_downfiltered = filter_flux(time, flux, freq_fft)

sampling_interval_downfiltered = 1.0/freq_sampling_downfiltered

In [ ]:
def calculate_window_function(t, fary):
    LSwindow = LombScargle(t, 1.0, fit_mean = False, center_data=False)
    Pwindow = LSwindow.power(fary)
    return LSwindow, Pwindow

In [ ]:
#https://github.com/jakevdp/PracticalLombScargle/blob/master/figures/StructuredWindows.ipynb
   
# WF for old time array
ls_window_old, p_window_old = calculate_window_function(old_time, freq_grid)

# WF for new time array
ls_window, p_window = calculate_window_function(time, freq_grid)

# WF for downfiltered (new) time array
ls_window_downfiltered, p_window_downfiltered = calculate_window_function(time_downfiltered, freq_grid)

# WF for evenly sampled data 
x_dt = np.arange(0, time_span, dt)
ls_window_dt, p_window_dt = calculate_window_function(x_dt, freq_grid)

fig, ax = plt.subplots(1,1, figsize=(16, 6)) 
ax.plot(freq_grid, p_window_old, label = "Raw Original Time")
ax.plot(freq_grid, p_window, c="green", ms=0.3, alpha=0.5, label = "Time (Gaps filled)")
ax.plot(freq_grid, p_window_dt, c="orange", ms=0.2, alpha=0.5, label = "Evenly sampled data")
ax.plot(freq_grid, p_window_downfiltered, c="purple", ms=0.1, alpha=0.5, label = "Downfiltered Time")
ax.set_xlabel(r"Frequency [$d^{-1}$]", size=25, labelpad=5)
ax.set_ylabel("Window Power", size=25, labelpad=10)
ax.legend(loc="upper right", fontsize=20)
ax.set_xlim(0,40)
ax.set_title("Periodogram of Observing Window", fontsize=30);

fig, ax = plt.subplots(2, figsize=(15, 10), sharex=True)
axs = [ax[0], ax[1]]
fig.subplots_adjust(hspace=0.35)
ax[0].plot(freq_grid, p_window, c="grey")
ax[0].set_ylabel('Window power', fontsize=20)
ax[0].set_title('Window Power Spectrum', fontsize=25);

ax[1].plot(ls_spec["periodogram"][0], ls_spec["periodogram"][1], "-k")
ax[1].set_title('Light Curve Power Spectrum', fontsize=25);
ax[1].set_ylabel("Lomb-Scargle power", fontsize=20)
for a in axs:
    a.set_xlim(0,50)
    a.set_xlabel('Frequency [d$^{-1}$]', fontsize=20)

## Create Spectrogram of Unevenly Sampled Data
<a class="anchor" id="section_2_3"></a>

The following is an implementation of a spectrogram that can deal with unevently sampled data. It uses the extended Lomb-Scargle periodogram.

1. Windows: nice article explaining some of the fundamental ones: https://community.sw.siemens.com/s/article/window-types-hanning-flattop-uniform-tukey-and-exponential Note on windows: Windows are used to "stretch" the frequency peaks. With spectrograms, one tries to overlap them so that each measurement is taken into account with the same weight as every other measurement.

2. dt2: (a) Too high time resolution (low Δ𝑡2 ) will result in fuzziness along the frequency direction (b) Too low time resolution (high Δ𝑡2 ) will result in the inability to capture frequencies changing too fast over time (c) trade off time resolution will give a well focused spectrogram with able to detect frequencies that change over tim

In [ ]:
def create_spectrogram(time, flux):
    
    #Create Tukey Window 
    alpha = 0.25      # "Shape parameter of the Tukey window, representing the fraction of the window inside the cosine tapered region. If zero, the Tukey window is equivalent to a rectangular window. If one, the Tukey window is equivalent to a Hann window."
    window = ("tukey", alpha)
    Nwindow = 101  # Nº of sample points to interpolate from the window
    N_samples_per_peak = 10 #shouldn't be less than 10
    
    print("* Generating a TUKEY window with...")
    print("     alpha = %0.02f" %alpha)
    print("     Nº of sample points to interpolate from window  = %i" %Nwindow)
    
    # Desired Time Resolution and Overlap
    dt2 = 0.16 
    overlap = alpha/2 #Currently set to 1/8, which is the default spectrogram value in python. 
    print("\n* Generating Spectrogram with...")
    print("     Desired Time Resolution = %0.02f " % dt2)
    print("     Desired Overlap = %0.02f" %overlap)
    
    #Max. Nº of samples per chunk for a given overlap
    Nt_chunk = dt2 / dt / (1 - overlap) # max. nº of samples per segment with an overlap equal to "overlap", if dt2 is to be respected
    Nt_chunk = max(int(2**np.floor(np.log2(Nt_chunk))), 2) # closer power of two below maximum. note: FFTs work best when N is a power of 2.
    print("\n* If Desired Time Resolution and Overlap is to be respected, we have...")
    print("     Nº of samples per window = %i" %Nt_chunk)
    
    #Time resolution of each window.
    effective_dt2 = (Nt_chunk - Nt_chunk//(1/overlap))*dt
    print("     Time resolution per window = %0.03f [d] (desired %0.03f [d])" %(effective_dt2, dt2))
    
    #Duration of each segment
    Tseg = dt * (Nt_chunk - 1) # Duration of one chunk (= duration of one window)
    print("     Duration of one window = %0.03f [d]. This yields an effective frequency resolution of %0.02f [d^-1]" %(Tseg, 1/Tseg))
    
    #Create Spectrogram Time Array
    #The variable `t2` is the center of each window. It goes from where the 1st time measurement is
    #plus T_seg/2 (half a window), to where the last time measurement is minus the region that is overlapped
    print("\n* Creating the Spectrogram TIME Array...")
    t2 = np.arange(time[0]+Tseg/2, time[-1]-Tseg/2, Tseg*(1-overlap))
    t_seg_start = t2 - Tseg/2
    t_seg_end   = t2 + Tseg/2
    
    #Create Spectrogram Frequency Array
    print("* Creating the Spectrogram FREQUENCY Array...")
    df2 = 1/(Tseg*N_samples_per_peak) #p.36 vanderplas
    grid_resolution_spectrogram = (maxfr-minfr)/df2+1
    f2 = np.linspace(minfr, maxfr, int(grid_resolution_spectrogram))
    
    #Prepare window to be applied to the data
    window_data = signal.get_window(window, Nwindow)
    window_t = np.arange(0, Nwindow) * Tseg/(Nwindow - 1)
    get_window_value = interpolate.interp1d(window_t, window_data)
    apply_window = lambda t, x: x*get_window_value(t)
    
    #Generate the Spectrogram
    P = np.zeros((f2.size, t2.size))
    print("* Generating the Spectrogram...")
    for i in range(t2.size):
        t_seg_flag = np.logical_and(time >= t_seg_start[i], time < t_seg_end[i])
        ind_seg = np.flatnonzero(t_seg_flag)

        if ind_seg.size < 2:  # No data in this segment, skip
            P[:,i] = np.nan
            continue

        t_seg = time[ind_seg] - t_seg_start[i]
        y_seg = flux[ind_seg]

        # Detrend
        y_seg = signal.detrend(y_seg, type="linear")

        # Window
        y_seg = apply_window(t_seg, y_seg) #Apply the given window to the given array along the given axis.

        # Lomb Scargle
        ls = LombScargle(t_seg, y_seg, normalization='psd')
        p = ls.power(f2)

        P[:,i] = p
    
    print("--> Done.")
    return P, t2, f2

In [ ]:
P, t2, f2 = create_spectrogram(time_osc, flux_osc)

f_min = min(freq_grid)
f_max = int(1.2*max(osc_data["freq"]))
f_range = f_max-f_min
pmin = 0.0
color_map ="magma" #"coolwarm" #"magma" #"coolwarm" #"Blues" #"RdYlBu_r" #"RdYlBu_r", "Greysd

fig, ax = plt.subplots(1,1, figsize=(15,10))
s = ax.pcolormesh(t2, f2, 10*np.log10(P), vmin=pmin,  cmap=color_map)# ="RdYlBu_r")
ax.set_ylabel(r'Frequency [$d^{-1}$]', size=30, labelpad=15)
ax.set_xlabel('Time [BKJD]', size=30, labelpad=15)
ax.set_ylim(min(freq_grid),f_max)
fig.colorbar(s, ax=ax, label = "Power [dB]")
ax.set_title(r"Spectrogram with f $>$ %i $d^{-1}$ masked out" %f_max,  y=1.02);

In [ ]:
widths = [5, 1.2] # two columns, with widths ratio 6 to 1
heights = [2, 6, 1, .4] # 4 columns

#   height of top plot, central plot, space for xlabels of central plot and colorbar below
fig, _ = plt.subplots(0, 0, figsize=(20,14))
grid = plt.GridSpec(
    4,
    2,
    hspace=0.05,
    wspace=0.02,
    figure=fig,
    width_ratios=widths,
    height_ratios=heights,
)
# Set up main plot: time vs freq and power as color
ax = fig.add_subplot(grid[1:-2, :-1])

# Set up top plot
ax_top = fig.add_subplot(grid[0, :-1])
ax_top.scatter(time, flux, c="lightsteelblue", s=1)  
ax_top.set_ylabel("Downfiltered Flux [ppt]", size=25)
ax_top.set_title("Spectrogram Input Data (Observed Flux - (Phase Curve + Eclipse Models)", size=22, y=1.05);

# Set up right plot: freq as y axis hence sharey. NOTE it's inverted wrt the figure above
ax_right = fig.add_subplot(grid[1:-2, -1], sharey=ax)
ax_right.plot(ls_spec["periodogram"][1], ls_spec["periodogram"][0], c="k") 
ax_right.set_xlabel("Power", size=25)
ax_right.set_ylim(f_min, f_max)
ax_right.set_title("LS Periodogram", size=28, y=1.02);

# Create axes for colorbar below
ax_bottom = fig.add_subplot(grid[-1, :-1])

# Personalize axes
ax_top.tick_params(labelbottom=False)
ax_right.tick_params(labelleft=False)

# main plot
mesh_power = ax.pcolormesh(t2, f2, 10*np.log10(P), vmin=pmin, cmap=color_map)
ax.set_ylabel(r'Frequency [$d^{-1}$]', size=25)
ax.set_xlabel("Time - %i [BKJD days]"%kepler_tref, size=25)
ax.set_ylim(f_min, f_max)
fig.colorbar(mesh_power, cax=ax_bottom, label = "Power [dB]", orientation="horizontal")

# Multi-Resolution Spectrogram 

Choose overlap and overlap factor. Notes: 
* Window length will be given by `n_overlap`*`overlap_factor`.
* n_overlap list should have two values, where the second value is half the first value.

In [ ]:
def plot_spectrogram(d_t, t_corners, f_corners, spectrogram_grid, t_lims = None, f_lims = None, name = None, variable = 'amp', units = 'physical', ft_mesh = None, show_mesh = False):
    # Note: Both amplitude and power get multiplied by one power of d_t to give appropriate units and correct limit as d_t -> 0.0.
    
    amplitude_spectrum      = np.abs(spectrogram_grid)*d_t
    power_spectrum          = (np.abs(spectrogram_grid)**2.0)*d_t

    assert variable in ['amp', 'power']
    assert units in ['physical', 'decibels']

    if variable == 'amp':
        if units == 'physical':
            Z = amplitude_spectrum
            Z_label = 'Amplitude (ppt day)'
            Z_lims = [1.0E-3, 2.0E-1]
            
        elif units == 'decibels':      
            Z0 = 1.0 # Reference amplitude in units of ppt day.
            Z = 20.0*np.log10(amplitude_spectrum/Z0)
            Z_label = 'Amplitude (20 $\log A/A_{0}$)'

    elif variable == 'power':
        if units == 'physical':
            Z = np.abs(spectrogram_grid)**2.0
            Z_label = 'Power (ppt$^{2}$ day)'
            Z_lims = [3.0E-3, 1.0E4]
            
        elif units == 'decibels':
            Z0 = 1.0 # Reference amplitude in units of (ppt^2) day.
            Z = 20.0*np.log10(power_spectrum/Z0)
            Z_label = 'Power (10 $\log_{10} P/P_{0})'

    #print(np.min(np.log10(Z)), np.max(np.log10(Z)))

    # Create the axes.
    fig = plt.figure(figsize = (11.0, 8.5))
    ax  = plt.gca()
    
    ## Find the colour-bar limits. Use a logarithmic colour-bar normalisation.
    c_norm = colors.LogNorm(vmin = Z_lims[0], vmax = Z_lims[1])

    if show_mesh: kwargs = {'edgecolor' : 'k'}
    else: kwargs = {}

    # Plot the spectrogram.
    image = ax.pcolormesh(t_corners, f_corners, Z.T, cmap = 'magma', norm = c_norm, **kwargs)

    #if ft_mesh is not None:
    #    for tc in ft_mesh[1]: ax.axvline(tc, alpha = 0.5, color = 'k')
    #    for fc in ft_mesh[0]: ax.axhline(fc, alpha = 0.5, color = 'k')

    # Tidy the axes.
    ax.set_xlabel('Time (days)', fontsize = font_size_label)
    ax.set_ylabel('Frequency (cycles per day)', fontsize = font_size_label)
    #
    if f_lims is not None: ax.set_ylim(f_lims)
    if t_lims is not None: ax.set_xlim(t_lims)

    # Add the colour bar.
    divider = make_axes_locatable(ax)
    c_ax = divider.append_axes('right', size = '2%', pad = 0.05)
    c_bar = fig.colorbar(image, cax = c_ax, orientation = 'vertical') 
    c_bar.ax.set_ylabel(Z_label, fontsize = font_size_label)

    # Tight layout.
    plt.tight_layout()

    # Save the figure.
    if name is not None:
        path_fig = os.path.join('plots', '{:}.png'.format(name))
        print('Saving to {:}'.format(path_fig))
        plt.savefig(path_fig)

    return

In [ ]:
def plot_window_function(f_win):
    fig = plt.figure(figsize = (12, 6))
    ax = plt.gca()
    ax.plot(f_win, marker = '.')
    ax.set_xlabel('Sample ID', fontsize=25)
    ax.set_ylabel('Window function', fontsize=25)
    plt.show()
    return

def plot_windows(i0, i1, n_samples):
    n_windows = len(i0)
    offsets = np.array(list(range(n_windows))) + 1.0

    fig = plt.figure(figsize = (12, 6))
    ax  = plt.gca()
    
    print('Window  First   Last')
    for j in range(n_windows):
        print('{:>5d} {:>5d} {:>5d}'.format(j, i0[j], i1[j]))
        i_window = np.array(list(range(i0[j], i1[j])), dtype = np.int)
        ax.scatter(i_window, np.zeros(len(i_window)) + offsets[j], s = 1, color = 'r') 
        
    #Each red line is one of the windows. These windows overlap with each other. 
    #The last window doesn't quite go to the end bc the length of the window is not a perfect multiple of the length of the timeseries

    ax.set_xlabel('Sample index')
    ax.set_ylabel('Window ID')
    ax.axvline(0)
    ax.axvline(n_samples - 1)
    buff = 0.05*i1[-1]
    ax.set_xlim([-buff, n_samples + buff])
    plt.show()
    return

In [ ]:
def create_cosine_window(length, edge_length):
    #We'll use a cosine window. we could also use a Tukey window
    
    # Interior part is just ones.
    f_win = np.zeros(length) + 1.0
    
    # Left edge is an increasing cosine taper.
    left_edge_pts = (np.pi)*np.linspace(-1.0, 0.0, num = edge_length)
    f_win[0 : edge_length] = np.cos(left_edge_pts)

    # Right edge is a decreasing cosine taper.
    right_edge_pts = (np.pi)*np.linspace(0.0, 1.0, num = edge_length)
    f_win[-edge_length : ] = np.cos(right_edge_pts)
    
    return f_win

In [ ]:
def spectrogram(x, d_t, n_samples_per_window, n_overlap, offset = 0, show_helper_plots = True):

    # Generate the window function.
    window_function = create_cosine_window(n_samples_per_window, n_overlap)

    # Plot the window function.
    if show_helper_plots: plot_window_function(window_function)

    # Calculate sampling frequency (note: assumes uniformly-spaced data).
    n_samples = len(x)
    time_span = (n_samples - 1)*d_t
    sampling_freq = 1.0/d_t

    # Calculate the number of windows that fit in the total dataset 
    shift_per_window = (n_samples_per_window - n_overlap)
    n_windows = ((n_samples - n_samples_per_window - offset)//shift_per_window) + 1
    
    # Note: Excess points at the end are not used. 
    # These two lines below don't affect the code, it's just for visualization purposes. 
    n_final = (n_windows - 1)*shift_per_window + n_samples_per_window + offset
    n_ignore = n_samples - n_final

    # Print some useful information.
    window_length = (n_samples_per_window - 1)*d_t
    overlap_length = (n_overlap - 1)*d_t
    print('Calculating spectrogram.')
    print('Total length of time series: {:>.3f} days ({:>5d} samples).'.format(time_span, n_samples))
    print('Window length: {:>.3f} days ({:>5d} samples).'.format(window_length, n_samples_per_window))
    print('Overlap:       {:>.3f} days ({:>5d} samples).'.format(overlap_length, n_overlap))
    print('Number of windows: {:>5d}'.format(n_windows))
    print('Samples skipped at beginning: {:>5d}.'.format(offset))
    print('Number of samples ignored at end: {:>5d}.'.format(n_ignore))

    # Get the start and end index of each window.
    i0 = np.array(list(range(n_windows)), dtype = np.int)*shift_per_window #start index
    i1 = i0 + n_samples_per_window
    #
    i0 = i0 + offset
    i1 = i1 + offset

    # Plot the windows.
    if show_helper_plots: plot_windows(i0, i1, n_samples)
        
    # Get the frequencies.
    freq = np.fft.rfftfreq(n_samples_per_window, d_t)
    n_freq = len(freq)
    
    # Get the times at the start, end, and middle of each window.
    t_mid = np.zeros(n_windows)
    t0 = np.zeros(n_windows)
    t1 = np.zeros(n_windows)
    for j in range(n_windows):
        t0[j] = d_t*i0[j] #start time
        t1[j] = t0[j] + window_length #end time
        t_mid[j] = (t0[j] + t1[j])/2.0 #midpoint of the time window

    
    # Initialise the spectrogram array.
    spectrogram = np.zeros((n_windows, n_freq), dtype = np.complex)
    for j in range(n_windows):    
        #print('Calculating spectrogram for window {:>5d} of {:>5d}'.format(j + 1, n_windows))
        x_windowed = window_function*(x[i0[j] : i1[j]])
        spectrogram[j, :] = np.fft.rfft(x_windowed) #we'll use real data, so that's why we are running rfft

    # Find the corners of (each pixel) of the frequency grid. 
    #this is only necessary for plotting purposes. 
    #the frequencies are evenly spaced (thanks to linearity of Fourier transform).
    freq_corners = np.zeros(n_freq + 1)
    freq_corners[1:-1] = freq[0:-1] + 0.5*np.diff(freq)
    freq_corners[-1] = freq[-1] + 0.5*(freq[-1] - freq[-2]) #the pixel will be the same size as the previous pixel

    # Find the corners of the time grid. 
    #Since there is a slight overlap between each window, we choose
    #the midpoint of the pixel as halfway through the two overlapping 
    #points of one window and the next. 
    t_corners = np.zeros(n_windows + 1)
    for j in range(1, n_windows):
        t_corners[j] = (t1[j - 1] + t0[j])/2.0
    
    t_corners[0] = t0[0] 
    t_corners[-1] = t_corners[-2] + (t_corners[-2] - t_corners[-3]) 

    return t0, t1, t_mid, t_corners, freq, freq_corners, spectrogram 

In [ ]:
def spectrogram_two_resolutions(x, d_t, n_samples_per_window_list, n_overlap_list, offset_list, merge_frequency, show_helper_plots = True):
    # Coarser times: t_corners_A.
    # Coarser freqs: f_corners_B.
    _, _, _, t_corners_A, _, f_corners_A, x_spectrogram_A = \
        spectrogram(x, d_t, n_samples_per_window_list[0], n_overlap_list[0],
                    offset = offset_list[0],
                    show_helper_plots = show_helper_plots)
    #
    _, _, _, t_corners_B, _, f_corners_B, x_spectrogram_B = \
        spectrogram(x, d_t, n_samples_per_window_list[1], n_overlap_list[1],
                    offset = offset_list[1],
                    show_helper_plots = show_helper_plots)

    # We now must move the spectrograms onto a finer grid, so that they can be merged. This involves 
    # increasing the time resolution of one of the spectrograms (by repeating elements) and increasing
    # the spectral resolution of both spectrograms (as their spectral grids will otherwise not match up).

    # Double the time resolution of the coarse time spectrogram.
    x_spectrogram_A_new = np.repeat(x_spectrogram_A, 2, axis = 0)
    
    # Fill any gaps at the end of the x-axis with zeros.
    n_t_missing = (len(t_corners_B) - x_spectrogram_A_new.shape[0] - 1)
    
    if n_t_missing > 0:
        x_spectrogram_A_new = np.pad(x_spectrogram_A_new, ((0, n_t_missing), (0, 0)))
    
    # Rename variables.
    t_corners_new = t_corners_B #time array for the merged spectrogram.  
    x_spectrogram_A_tmp = x_spectrogram_A_new.copy()

    # Determine the new master frequency grid.
    f_corner_max = f_corners_B[-1]
    n_f_corners_new = int(round((f_corner_max/(np.median(np.diff(f_corners_A)))))) + 1
    n_f_corners_new = 2*n_f_corners_new - 1
    f_corners_new = np.linspace(0.0, f_corner_max, num = n_f_corners_new)

    # Initialise re-gridded spectrograms.
    n_t_new = x_spectrogram_A_tmp.shape[0]
    x_spectrogram_A_new = np.zeros((n_t_new, n_f_corners_new - 1), dtype = np.complex)
    x_spectrogram_B_new = np.zeros((n_t_new, n_f_corners_new - 1), dtype = np.complex)

    # To resample high-fuency version, each element must be repeated twice (except first element).
    # First element is not repeated.
    x_spectrogram_A_new[:, 0] = x_spectrogram_A_tmp[:, 0]
    
    # Repeat the elements.
    x_spectrogram_A_new[:, 1:-1] = np.repeat(x_spectrogram_A_tmp[:, 1:], 2, axis = 1)

    # To re-sample low-fuency version, each element must be repeated four times (except first element).
    # First element is repeated only once.
    x_spectrogram_B_new[:, 0] = x_spectrogram_B[:, 0]
    x_spectrogram_B_new[:, 1] = x_spectrogram_B[:, 0]

    # Repeat the elements.
    x_spectrogram_B_new[:, 2:] = np.repeat(x_spectrogram_B[:, 1:], 4, axis = 1)

    # Merge the spectrograms.
    i_f_switch = np.argmax(f_corners_new > merge_frequency)
    x_spectrogram_merged = np.zeros(x_spectrogram_A_new.shape, dtype = np.complex)
    x_spectrogram_merged[:, 0 : i_f_switch] = x_spectrogram_A_new[:, 0 : i_f_switch]
    x_spectrogram_merged[:, i_f_switch : ] = x_spectrogram_B_new[:, i_f_switch : ]

    return  t_corners_A,        f_corners_A,        x_spectrogram_A, \
            t_corners_B,        f_corners_B,        x_spectrogram_B, \
            t_corners_new,      f_corners_new,      x_spectrogram_merged

The 1st spectrogram has a bigger window length. It is used for the lower part of the spectrogram, which is where the shorter frequencies are (they have longer cycles).  the shorter frequencies. The 2nd spectrogram has a shorter window length. In the following arrays, the 1st and 2nd item refer to the 1st and 2nd spectrogram, respectively. We've chosen the window length of one spectrogram to be half of the other one. 

In [ ]:
overlap_factor = 8 #what fraction of the window overlaps with the previous windows (1/8)
n_overlap_bigger = 128 #number of samples that overlap with the previous one (128 is quite big) -> the window length will be 8*128, which is 1024
n_overlap_list = [n_overlap_bigger, n_overlap_bigger//2] #the 1st and 2nd item is for the 1st and 2nd spectrogram, respectively
offset_list = [0, n_overlap_list[1]//2] #the 0 is for the bigger window, and the 2nd item is for the smaller window.  
n_samples_per_window_list = [overlap_factor*n_overlap for n_overlap in n_overlap_list]

In [ ]:
# Calculate the two spectrograms and merge them.
show_helper_plots = True
merge_frequency = 30.0 # Cycles per day. this is the cut-off at which the transition takes place
t_corners_A,        f_corners_A,        flux_spectrogram_A, \
t_corners_B,        f_corners_B,        flux_spectrogram_B, \
t_corners_merged,   f_corners_merged,   flux_spectrogram_merged, \
= spectrogram_two_resolutions(flux, dt, n_samples_per_window_list,
                              n_overlap_list, offset_list, merge_frequency,
                              show_helper_plots = show_helper_plots)

In [ ]:
# Plot the merged and un-merged spectrograms.
font_size_label = 25
common_args = { 'variable' : 'amp', 'units' : 'physical',
               't_lims' : [0.0, np.max(time) - np.min(time)],
               'f_lims' : [0.0, 60.0]}

name_list = ['spectrogram_{:>05d}'.format(n_samples_per_window) for n_samples_per_window in n_samples_per_window_list]
plot_spectrogram(dt, t_corners_A, f_corners_A, flux_spectrogram_A, **common_args) #, name = name_list[0])
plot_spectrogram(dt, t_corners_B, f_corners_B, flux_spectrogram_B, **common_args)# name = name_list[1])
plot_spectrogram(dt, t_corners_merged, f_corners_merged, flux_spectrogram_merged, **common_args) #, name = 'spectrogram_merged')
plt.show()

In [ ]:
tperiastron = pd.read_csv("KIC%s/%s/data/t_periastron.txt" %(KIC, segment), header=0).values[0][0]
t0_secondary = pd.read_csv("KIC%s/%s/data/t0_secondary.txt" %(KIC, segment), header=0).values[0][0]

def calculate_n_integer(t, t_ref, P):
    min_n = round((min(t)-t_ref)/P)
    max_n = round((max(t)-t_ref)/P)+1
    n = np.arange(min_n, max_n, 1)
    return n

n_periastron = calculate_n_integer(time, tperiastron, lit_period)
n_peclipses = calculate_n_integer(time, lit_t0, lit_period)
n_seclipses = calculate_n_integer(time, t0_secondary, lit_period)

periastron_passages = tperiastron + n_periastron*lit_period
primary_passages = lit_t0+n_peclipses*lit_period
secondary_passages = t0_secondary+n_seclipses*lit_period

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,6))
ax.scatter(time, flux, c="lightsteelblue", s=1)
for i,(peri, prim, sec) in enumerate(zip(periastron_passages, primary_passages, secondary_passages)):
    if i==0:
        label1 = "Periastron"
        label2 = "Primary"
        label3 = "Secondary"
    else:
        label1, label2, label3 = None, None, None
    ax.axvline(peri, lw=2, c="r", label=label1)
    ax.axvline(prim, lw=2, c="g", label=label2)
    ax.axvline(sec, lw=2, c="y", label=label3)
ax.legend(fontsize=20)    
ax.set_xlabel("Time - %i [BKJD days]"%kepler_tref, fontsize=20)
ax.set_ylabel("Raw Flux (ppt)", fontsize=20);
ax.set_xlim(min(time), max(time)) ;

In [ ]:
def show_eclipses_and_periastron(t, y, Tperiastron, Tprimary, Tsecondary, ax = ax):
    #ax.scatter(t, y, s=1, c="lightsteelblue")
    
    for i,(peri, prim, sec) in enumerate(zip(Tperiastron, Tprimary, Tsecondary)):
        if i==0:
            label1 = "Periastron"
            label2 = "Primary"
            label3 = "Secondary"
        else:
            label1, label2, label3 = None, None, None
        
        ax.axvline(peri, lw=2, c="r", label=label1)
        ax.axvline(prim, lw=2, c="g", label=label2)
        ax.axvline(sec, lw=2, c="y", label=label3)
    
    ax.legend(fontsize=15)    
    ax.set_ylabel("Raw Flux (ppt)", fontsize=20);
    ax.set_xlabel("Time - %i [BKJD days]"%kepler_tref, fontsize=20)
    
    return 

# Wavelet Analysis

References:
* <a href="https://www.kaggle.com/asauve/a-gentle-introduction-to-wavelet-for-data-analysis"> [Kaggle]</a>, https://www.kaggle.com/mistag/extracting-bird-song-signatures-with-wavelets
* <a href="https://www.mathworks.com/help/signal/ug/scalogram-computation-in-signal-analyzer.html"> [Source of Text below]</a>
* <a href="https://es.mathworks.com/help/wavelet/gs/continuous-wavelet-transform-and-scale-based-analysis.html"> [Mathworks]

The *spectrogram* is obtained by windowing the input signal with a WINDOW of constant length (duration) that is shifted in time and frequency. The window used in the spectrogram is even, real-valued, and does not oscillate. Because the spectrogram uses a constant window, the time-frequency resolution of the spectrogram is fixed.

By contrast, the *CWT* is obtained by windowing the signal with a WAVELET that is scaled and shifted in time. The wavelet oscillates and can be complex-valued. The scaling and shifting operations are applied to a prototype wavelet. The scaling used in the CWT both shrinks and stretches the prototype wavelet. Shrinking the prototype wavelet yields short duration, high-frequency wavelets that are good at detecting transient events. Stretching the prototype wavelet yields long duration, low-frequency wavelets which are good at isolating long-duration, low frequency events.


In [ ]:
import pywt
import scaleogram as scg

Removing the mean value is mandatory, otherwise the borders of the data are considered as steps which create a lot of spurious cone shaped detection. https://www.kaggle.com/asauve/a-gentle-introduction-to-wavelet-for-data-analysis

In [ ]:
def normalize(d):
    #NORMALIZE FUNCTION by - mean/sqrt(variance)
    mean = np.mean(d)
    variance = np.var(d)
    d = (d - mean) / (np.sqrt(variance))
    print("mean = ", mean)
    print("std = ", np.sqrt(variance))
    print("Mean Absolute Deviation = %0.02f" % madev(d))
    return d

def madev(d, axis=None):
    """ Mean absolute deviation of a signal """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)


In [ ]:
flux_norm = normalize(flux)

fig, ax = plt.subplots(1,1)
ax.scatter(time, flux_norm, s=1, c="navy")
ax.set_title("Flux Normalized by mean/sqrt(variance)", y=1.02, fontsize=20);

Scaleogram Parameters

In [ ]:
chosen_wavelet = 'cmor2-6' #choice of 6th order morlet with B=2 taken after looking at various morlet wavelets and evaluating tradeoff (post discussion with Daniel del Ser and Octavi)
scg.set_default_wavelet(chosen_wavelet)
fig, ax = plt.subplots(1,2, figsize=(16,6))
scg.plot_wav(chosen_wavelet, axes=ax, real=True, imag=True, yscale="linear")
central_frequency = pywt.central_frequency(chosen_wavelet)
ax[0].set_ylim(-0.6, 0.6);
print(central_frequency)

Some notes related to the code in the cell above:

* Scales: With logscale on Y axis, the bandwith will have the same height at all scales which may be helpful for data interpretation. From https://paos.colorado.edu/research/wavelets/faq.html : "The scale refers to the width of the wavelet. As the scale increases and the wavelet gets wider, it includes more of the time series, and the finer details get smeared out. The scale can be defined as the distance between oscillations in the wavelet (e.g. for the Morlet), or it can be some average width of the entire wavelet (e.g. for the Marr or Mexican hat). The period (or inverse frequency) is the approximate Fourier period that corresponds to the oscillations within the wavelet. For all wavelets, there is a one-to-one relationship between the scale and period. The relationship can be derived by finding the wavelet transform of a pure cosine wave with a known Fourier period, and then computing the scale at which the wavelet power spectrum reaches its maximum. For some wavelets the period has more meaning than others. For the Morlet, which has several smooth oscillations, the period is a well-defined quantity which measures the approximate Fourier period of the signal. For the Daubechies, which has irregularly-spaced oscillations, the period has less meaning and should probably be ignored."

* Wavelet Choice: We choose the Morlet wavelet because: (1) it is commonly used, (2) it's simple, (3) it looks like a wave <a href="https://paos.colorado.edu/research/wavelets/wavelet3.html">[ref]</a>

In [ ]:
min_period_int = 0.1
max_period_int = 700
periods_wavelet = dt*np.logspace(np.log10(min_period_int), np.log10(max_period_int + 1), 1500)  
scales  = scg.periods2scales(
                            periods_wavelet,
                            wavelet = chosen_wavelet,
                            dt      = dt) #sampling interval

#print(central_frequency*periods_wavelet/(scales*dt))

In [ ]:
def cyclesday_to_hertz(c):
    return c*1.157*10**(-5)

def scale_to_fourierFrequency(s):
    #This applies to Morlet Wavelet
    #See table 1 https://cobblab.eas.gatech.edu/seminar/torrence&compo98.pdf
    num = 4*np.pi*s
    den = central_frequency + np.sqrt(2+central_frequency**2)
    f = num/den
    return f

print("Min. Frequency = %0.02f [c/d]" %(minfr))
print("\t If %0.02f [c/d] is chosen as min. scale, the corresponding Fourier frequency is f=%0.02f [c/d]" %(minfr, scale_to_fourierFrequency(minfr)))

print("Max Frequency = %0.02f [c/d] (f_nyquist)" %(f_nyquist))
print("\t If %0.02f [c/d] is chosen as max. scale, the corresponding Fourier frequency is f=%0.02f [c/d]" %(f_nyquist, scale_to_fourierFrequency(f_nyquist)))

In [ ]:
#Original code: https://github.com/chris-torrence/wavelets/blob/master/wave_python/waveletAnalysis.py
       
def calculate_scaleogram(t, f, scale_ary, 
                         y_axis           = "frequency", 
                         wavelet_type     = chosen_wavelet, 
                         y_scale          = "log", 
                         c_map            = "plasma",  #jet
                         spectrum_type    = "amp",
                         ax               = None,
                         c_lim            = None,
                         y_lim            = [minfr, f_max],
                         x_label          = "Time/Spatial Domain",
                         y_label          = None,
                         fig_title        = "Continuous Wavelet Transform Amplitude Spectrum",
                         name             = None):
    
    print("Computing the CWT with a %s mother wavelet..." %wavelet_type)
    
    ax = scg.cws(t, 
             signal  = f,
             scales  = scale_ary,
             yaxis   = y_axis,
             wavelet = wavelet_type, 
             yscale  = y_scale, #To be able to see the small periods and the large ones at once, it is better to use logarithmic scale on the Y axis. This is achieved with the yscale='log'option.
             ax      = ax, 
             cmap    = c_map,
             spectrum = spectrum_type,
             clim    = c_lim
    ) 
    
    if y_label: ax.set_ylabel(y_label, fontsize=25)
    
    ax.set_title(fig_title, fontsize=28, y=1.02)
    
    if y_axis=="frequency":
        ax.set_yticks([2, 3, 4, 6, 8, 10, 20, 30, 40, 50, 60]) #yaxis ticks in scaleograms
        ax.set_ylim(y_lim)
    ax.get_yaxis().set_major_formatter(ScalarFormatter())
    ax.set_xlabel(x_label, fontsize=25)
    
    if name:
        plt.tight_layout()
        fig.savefig("KIC%s/%s/tfreq_analysis/%s.png" %(KIC, segment,name), dpi=300)
    return ax

Let's plot the CWT 

* Cone of Influence: Represents the locations where the data is strongly affected by border effects due to the extent in time of the child wavelet. As a rule of thumb, you can use the COI as a visual marker to appreciate the precision in time at a given scale. <a href="https://github.com/alsauve/scaleogram/blob/master/doc/scale-to-frequency.ipynb"> [Ref] </a>

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,8))
ax_spg = calculate_scaleogram(
    time, 
    flux_norm, 
    scales, 
    ax            = ax,
    wavelet_type  = chosen_wavelet,
    c_lim         = (0,8), 
    c_map         = "jet",
    x_label       = "Time [d]", 
    y_label       = r"Frequency $d^{-1}$",
    #name          = "full_cwt"
)

fig.tight_layout()
fig.savefig("KIC%s/frequency_analysis/full_cwt.png" %KIC, dpi=300)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,8))
fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
#scales = scg.periods2scales(np.logspace(np.log10(100), np.log10(f_nyquist), 500))
ax_spg = calculate_scaleogram(
    time, flux_norm, scales, 
    ax           = ax, 
    y_lim        = [minfr, 6],
    wavelet_type = chosen_wavelet,
    c_lim        = (0,8), 
    c_map        = "jet",
    x_label      = "Time [d]", 
    y_label      = r"Frequency $d^{-1}$",
)
ax.set_ylim([minfr, 6])
fig.tight_layout()
fig.savefig("KIC%s/frequency_analysis/zoom_cwt_lowfreqs.png" %KIC, dpi=300)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,8))
#scales = scg.periods2scales(np.logspace(np.log10(1/minfr), np.log10(100), 800))
ax_spg = calculate_scaleogram(
    time, 
    flux_norm, 
    scales, 
    ax           = ax, 
    y_lim        = [35, f_max],
    wavelet_type = chosen_wavelet,
    c_lim        = (0,8), 
    c_map        = "jet",
    x_label      = "Time [d]", 
    y_label      = r"Frequency $d^{-1}$",
)
fig.tight_layout()
fig.savefig("KIC%s/frequency_analysis/zoom_cwt_highfreqs.png" %KIC, dpi=300)

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(14,10))
ax[0].scatter(time, flux_norm, s=2, c="navy")
ax_spg = calculate_scaleogram(
    time, 
    flux_norm, 
    scales, 
    ax           = ax[1], 
    y_lim        = [f_min, f_max],
    wavelet_type = chosen_wavelet,
    c_lim        = (0,8), 
    c_map        = "jet",
    x_label      = "Time [d]", 
    y_label      = r"Frequency $d^{-1}$",
)
ax[0].set_xlim([305, 310])
ax[1].set_xlim([305, 310])
fig.tight_layout()
fig.savefig("KIC%s/frequency_analysis/perhaps_data_gap.png" %KIC, dpi=300)

## Plot the Retrieved Oscillations

In [ ]:
osc_data = osc_data.sort_values(by=['freq'], ascending=True).reset_index(drop=True)
osc_data.to_csv("KIC%s/frequency_analysis/retrieved_osc.csv" %KIC, sep=",", header=True, columns=osc_data.columns, index=False)

mask = osc_data.d_to_int<0.01
teo = osc_data[mask]
teo.sort_values(by=['freq'], ascending=True)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(16,8))
ax_spg = calculate_scaleogram(
    time, flux_norm, scales, 
    ax           = ax, 
    y_axis       = "frequency",
    #y_lim        = [minfr, 6],
    wavelet_type = chosen_wavelet,
    c_lim        = (0,10), 
    c_map        = "jet",
    x_label      = "Time [d]", 
    y_label      = r"Frequency [$d^{-1}$]",
)
for t in teo.freq.values:
    print(t)
    ax.axhline(t, color="white", lw=3, ls="-", alpha=0.6)
    
fig.tight_layout()
fig.savefig("KIC%s/frequency_analysis/full_cwt_retrieved_oscillations.png" %KIC, dpi=300)

# Tests with Wavelet Transforms

### Do a CWT on Phase-Folded Data

In [ ]:
def fold_data(ary, ntimes, P_ref, t0_ref):
    nperiod = ntimes*P_ref
    ary_fold = ((ary - t0_ref + 0.5 * nperiod) % nperiod)/nperiod - 0.5
    return ary_fold 

#Phase-fold times
#periastron_passages_fold = fold_data(periastron_passages, 1, lit_period, lit_t0)
#primary_passages_fold = fold_data(primary_passages, 1, lit_period, lit_t0)
#secondary_passages_fold = fold_data(secondary_passages, 1, lit_period, t0_secondary)
x_fold = fold_data(x, 1, lit_period, lit_t0)
time_fold = fold_data(time, 1, lit_period, lit_t0)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 7))
ax.scatter(x_fold, y, s=1, label="Folded Raw Light Curve")
ax.scatter(time_fold, flux_norm, s=1, label = "Folded Osc. Model + Noise. Model")
ax.set_xlim([-0.5, 0.5])
ax.set_ylim([-40, 20])
ax.set_xlabel("Orbital Phase", size=30)
ax.set_ylabel('Raw Flux [ppt]', size=30)
ax.legend(fontsize=18, markerscale=5)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,8))
calculate_scaleogram(time_fold, 
                     flux_norm, 
                     scales, 
                     wavelet_type   = chosen_wavelet,
                     ax             = ax, 
                     c_lim          = (0,10),
                     x_label        = "Phase [d]", 
                     y_label        = r"Frequency $d^{-1}$",
                     y_ticks        = kic_ticks,
                     
                    )
fig.tight_layout()
fig.savefig("phasefolded_cwt.png")

### Noise with Injected Low SNR signal

In [ ]:
noise_model = read_txt("KIC%s/%s/tfreq_analysis/noise_model.txt" %(KIC, segment), header=1, names=["Time", "Flux"])

In [ ]:
ls_noise = ls_estimator(noise_model.Time, noise_model.Flux, freq_grid, normalization="psd")

fig, ax = plt.subplots(1,1, figsize=(15,6))
ax.plot(ls_spec["periodogram"][0], ls_spec["periodogram"][1], alpha=0.5, label="Raw LC")
ax.plot(ls_noise["periodogram"][0], ls_noise["periodogram"][1], label="Best-fit Noise Model")
ax.set_xlabel(r"Frequency [$d^{-1}$]", size=20)
ax.set_ylabel("Power")
ax.set_title("LS Periodogram")
ax.legend(fontsize=15)
ax.set_xlim(0,f_max);
ax.text(0.1,0.6, "Low Frequency g-mode regime", fontsize=15, transform=ax.transAxes, bbox=dict(facecolor='red', alpha=0.4));
ax.text(0.62,0.6, "High Frequency p-mode regime", fontsize=15, transform=ax.transAxes, bbox=dict(facecolor='green', alpha=0.4));

In [ ]:
def noisify(sig, noise_amp=1):
    return sig + (np.random.random(len(sig))-0.5)*2*noise_amp

In [ ]:
x_fake = np.linspace(min(noise_model.Time), max(noise_model.Time), len(noise_model.Time))
amplitude_fake = 0.2
freq_fake = 10.0
theta = 0 # amplitude of our sine wave at time 0
y_fake = amplitude_fake * np.sin(2 * np.pi * freq_fake * x_fake + theta)
y_fake_noisy = noisify(y_fake, noise_amp=1)

fig, ax = plt.subplots(1,1, figsize=(16,6))
ax.scatter(x_fake, y_fake, s=1, c="dodgerblue", label="Fake Signal") #r"f$_{fake}$=%0.1f [c/d], A=%0.02f [ppt]" %(freq_fake, amplitude), c="salmon")
ax.scatter(x_fake, y_fake_noisy, s=0.5, alpha=0.5, c="green", label="Fake Signal with White Noise") 
ax.scatter(x_fake, noise_model.Flux+y_fake_noisy, s=1, color="orange", alpha=0.5, label="Original signal + Fake signal with White Noise")#  label=r"f$_{fake}$=%0.1f [c/d], A=%0.02f [ppt]" %(freq_fake, amplitude), c="salmon")
ax.plot(noise_model.Time, noise_model.Flux, "k.", ms=0.5, label="Noise Best-fit model")

ax.legend(fontsize=15, markerscale=4)
ax.set_title("Best-fit Noise Model + Injected Signal", y=1.02, fontsize=25)
ax.set_xlabel("Time [d]", fontsize=20);
ax.set_ylabel("Raw flux [ppt]", fontsize=20);

fig, ax = plt.subplots(1,1, figsize=(16,6))
calculate_scaleogram(x_fake, 
                     noise_model.Flux+y_fake_noisy, 
                     scales, 
                     wavelet_type  = chosen_wavelet, 
                     x_label       = "Time [d]", 
                     y_label       = r"Frequency $d^{-1}$",
                     ax = ax, c_lim=(0,8))
#fig.savefig("scg_noise_f%ia%0.01f.png" %(freq_fake,  amplitude_fake), dpi=150)

Note: Noise displays usually as a patchwork of features and some bumps can look like real data features, so you have to be careful when using real data to check the level of noise when appropriate. I recommend this fairly complete paper on the significance level by Z. Ge. if you have some time. (https://www.kaggle.com/asauve/a-gentle-introduction-to-wavelet-for-data-analysis)


In [ ]:
x_fake = np.linspace(min(noise_model.Time), max(noise_model.Time), len(noise_model.Time))
amplitude_fake = [0.2, 0.5]
freq_fake = [4.0, 30]
theta = [0, 0] # amplitude of our sine wave at time 0

allwaves, y_fake, y_fake_noisy = [], [], []
damping_coefficent = 0.1 

fig, ax = plt.subplots(1,1, figsize=(16,8))

for i in range(0,len(amplitude_fake)):
    y_fake = amplitude_fake[i] * np.sin(2 * np.pi * freq_fake[i] * x_fake + theta[i])
    y_fake_noisy = noisify(y_fake, noise_amp=1)
    allwaves.append(y_fake_noisy)

multiple_waves = np.copy(noise_model.Flux)
for w in allwaves: multiple_waves+=w
    
scales = scg.periods2scales(np.logspace(np.log10(0.1), np.log10(f_nyquist), 600))
calculate_scaleogram(x_fake, 
                     multiple_waves, 
                     scales, 
                     wavelet_type  = chosen_wavelet, 
                     ax            = ax, 
                     c_lim         = (0,10),
                     y_lim         = (f_min, f_max),
                     y_ticks       = kic_ticks, 
                     x_label       = "Time [d]", 
                     y_label       = r"Frequency $d^{-1}$")

#fig.savefig("scg_noise_allwaves.png")

In [ ]:
test_wavelets = {"cmor1-1":    "B=1.0, CF=1.0",
                 "cmor1_3":    "B=1.0, CF=3.0" ,
                 "cmor1-5":    "B=1.0, CF=5.0",
                 "cmor1-6":    "B=1.0, CF=6.0",
                 "cmor2-6":    "B=2.0, CF=6.0",
                 "cmor1-8":    "B=1.0, CF=8.0"}

nwavelets = len(test_wavelets.keys())
fig, ax = plt.subplots(nwavelets,1, figsize=(15,35), sharex=True)
fig.subplots_adjust(hspace=0.3)
scales = scg.periods2scales(np.logspace(np.log10(1/minfr), np.log10(f_nyquist), 800))

for i, (w, title) in enumerate(test_wavelets.items()):
        calculate_scaleogram(
        time, flux_norm, scales, 
        wavelet_type    = w, 
        x_label         = None, 
        y_label         = r"Frequency [$d^{-1}$]", 
        #y_lim           = [minfr, 10],
        y_ticks         = kic_ticks, 
        fig_title       = title,
        ax              = ax[i],
        c_map           = "jet",
        c_lim           = (0,8)
) 
fig.tight_layout()
fig.savefig("cmorlet_types.png", dpi = 300)

In [ ]:
theta = 0
amplitude_fake = 0.5#[0.2, 0.3, 1, 0.8]
freq_fake = 5
damped_signal = (amplitude_fake*(1-0.8*np.sin(2*np.pi/50*freq_fake*x_fake + theta)))* np.sin(2*np.pi *freq_fake* x_fake + theta)
damped_signal_noisy = noisify(damped_signal, noise_amp=0.1)

fig, ax = plt.subplots(2,1, figsize=(16,13))
fig.subplots_adjust(hspace=0.25)
#ax[0].scatter(noise_model.Time, noise_model.Flux, label="Noise model from Fit", s=1)
ax[0].scatter(x_fake, damped_signal_noisy, label="Fake Signal with Noise", s=1)
ax[0].legend(fontsize=18, markerscale=3)
ax[0].set_xlabel("Time [d]"); 
ax[0].set_ylabel("Raw Flux [ppt]")

'''
###### Best-fit noise model and noisy signal
calculate_scaleogram(noise_model.Time.values, noise_model.Flux+damped_signal_noisy, 
                     scales, 
                     wavelet_type="cmor1.5-6.0", 
                     c_lim         = (0, 10),
                     y_lim         = (f_min, f_max), 
                     ax            = ax[1], 
                     x_label       = "Time [d]", 
                     y_label       = r"Frequency $d^{-1}$",
                     fig_title     = None)

#fig.savefig("modulating_amplitude_noNoise.png")
fig.savefig("modulating_amplitude_Noise.png")
'''

###### Pure Noisy Signal
calculate_scaleogram(noise_model.Time.values, 
                     damped_signal_noisy, 
                     scales, 
                     wavelet_type  = chosen_wavelet, 
                     c_lim         = (0, 10),
                     y_lim         = (f_min, f_max), 
                     ax            = ax[1], 
                     x_label       = "Time [d]", 
                     y_label       = r"Frequency $d^{-1}$",
                     fig_title     = None)
fig.savefig("puresignal_cwt.png")